In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
import matplotlib
import pandas as pd
import datetime

In [2]:
#read data set
X=pd.read_csv("final1.csv")

In [3]:
X.head(10)

,token,task,age,gender,dept,doctortype,in_time,out_time
0,1,checkup,15,f,General Medicine,physician,12/28/2019 10:00,12/28/2019 10:03
1,1,billing,15,f,cashier,unidentifed,12/28/2019 10:04,12/28/2019 10:05
2,1,medicine,15,f,pharmacy,unidentifed,12/28/2019 10:06,12/28/2019 10:08
3,2,checkup,21,m,Surgery,surgeon,12/28/2019 10:04,12/28/2019 10:08
4,2,ct-scan,21,m,ct_lab,unidentifed,12/28/2019 10:09,12/28/2019 10:23
5,2,billing,21,m,cashier,unidentifed,12/28/2019 10:13,12/28/2019 10:15
6,2,medicine,21,m,pharmacy,unidentifed,12/28/2019 10:15,12/28/2019 10:17
7,3,checkup,45,m,General Medicine,physician,12/28/2019 10:05,12/28/2019 10:09
8,3,billing,45,m,cashier,unidentifed,12/28/2019 10:10,12/28/2019 10:11
9,3,medicine,45,m,pharmacy,unidentifed,12/28/2019 10:12,12/28/2019 10:13


In [4]:
X.describe()

,token,age
count,708.000000,708.000000
mean,17.440678,38.533898
std,9.833012,15.691518
min,1.000000,8.000000
25%,9.000000,27.000000
50%,17.000000,38.000000
75%,26.000000,49.000000
max,34.000000,70.000000


In [5]:
X.dtypes

token          int64
task          object
age            int64
gender        object
dept          object
doctortype    object
in_time       object
out_time      object
dtype: object

In [6]:
# convert string to datetime data type
X["in_time"] = pd.to_datetime(X.in_time)
X["out_time"] = pd.to_datetime(X.out_time)

X.dtypes

token                  int64
task                  object
age                    int64
gender                object
dept                  object
doctortype            object
in_time       datetime64[ns]
out_time      datetime64[ns]
dtype: object

In [7]:
X['dayofweek'] =X.in_time.dt.weekday

In [8]:
X.head()

,token,task,age,gender,dept,doctortype,in_time,out_time,dayofweek
0,1,checkup,15,f,General Medicine,physician,2019-12-28 10:00:00,2019-12-28 10:03:00,5
1,1,billing,15,f,cashier,unidentifed,2019-12-28 10:04:00,2019-12-28 10:05:00,5
2,1,medicine,15,f,pharmacy,unidentifed,2019-12-28 10:06:00,2019-12-28 10:08:00,5
3,2,checkup,21,m,Surgery,surgeon,2019-12-28 10:04:00,2019-12-28 10:08:00,5
4,2,ct-scan,21,m,ct_lab,unidentifed,2019-12-28 10:09:00,2019-12-28 10:23:00,5


In [9]:
#creating new column to check weekend / weekday
X["is_weekday"]=0

for i in range(0,len(X)):
    if ((X["dayofweek"][i]==5)|(X["dayofweek"][i]==6)):
        X["is_weekday"][i]=0
    else:
        X["is_weekday"][i]=1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
#creating new column to check morning/evening slot
X=X.assign(session=pd.cut(X.in_time.dt.hour,[0,13,24],labels=['Morning','Evening']))

In [11]:
X.head(100)

,token,task,age,gender,dept,doctortype,in_time,out_time,dayofweek,is_weekday,session
0,1,checkup,15,f,General Medicine,physician,2019-12-28 10:00:00,2019-12-28 10:03:00,5,0,Morning
1,1,billing,15,f,cashier,unidentifed,2019-12-28 10:04:00,2019-12-28 10:05:00,5,0,Morning
2,1,medicine,15,f,pharmacy,unidentifed,2019-12-28 10:06:00,2019-12-28 10:08:00,5,0,Morning
3,2,checkup,21,m,Surgery,surgeon,2019-12-28 10:04:00,2019-12-28 10:08:00,5,0,Morning
4,2,ct-scan,21,m,ct_lab,unidentifed,2019-12-28 10:09:00,2019-12-28 10:23:00,5,0,Morning
...,...,...,...,...,...,...,...,...,...,...,...
95,28,fracture,36,m,General Medicine,orthopedic,2019-12-28 19:43:00,2019-12-28 20:03:00,5,0,Evening
96,28,billing,36,m,cashier,unidentifed,2019-12-28 20:05:00,2019-12-28 20:07:00,5,0,Evening
97,28,medicine,36,m,pharmacy,unidentifed,2019-12-28 20:08:00,2019-12-28 20:10:00,5,0,Evening
98,29,checkup,8,f,General Medicine,physician,2019-12-28 19:53:00,2019-12-28 19:57:00,5,0,Evening


In [12]:
#creating age category
bins=[0,15,55,95]
group=['child','adult','old']
X['age_category']=pd.cut(X["age"],bins, labels=group)

In [13]:
X.head(50)

,token,task,age,gender,dept,doctortype,in_time,out_time,dayofweek,is_weekday,session,age_category
0,1,checkup,15,f,General Medicine,physician,2019-12-28 10:00:00,2019-12-28 10:03:00,5,0,Morning,child
1,1,billing,15,f,cashier,unidentifed,2019-12-28 10:04:00,2019-12-28 10:05:00,5,0,Morning,child
2,1,medicine,15,f,pharmacy,unidentifed,2019-12-28 10:06:00,2019-12-28 10:08:00,5,0,Morning,child
3,2,checkup,21,m,Surgery,surgeon,2019-12-28 10:04:00,2019-12-28 10:08:00,5,0,Morning,adult
4,2,ct-scan,21,m,ct_lab,unidentifed,2019-12-28 10:09:00,2019-12-28 10:23:00,5,0,Morning,adult
5,2,billing,21,m,cashier,unidentifed,2019-12-28 10:13:00,2019-12-28 10:15:00,5,0,Morning,adult
6,2,medicine,21,m,pharmacy,unidentifed,2019-12-28 10:15:00,2019-12-28 10:17:00,5,0,Morning,adult
7,3,checkup,45,m,General Medicine,physician,2019-12-28 10:05:00,2019-12-28 10:09:00,5,0,Morning,adult
8,3,billing,45,m,cashier,unidentifed,2019-12-28 10:10:00,2019-12-28 10:11:00,5,0,Morning,adult
9,3,medicine,45,m,pharmacy,unidentifed,2019-12-28 10:12:00,2019-12-28 10:13:00,5,0,Morning,adult


In [14]:
#calculating treatment time in seconds(integer) because ml doesnot work on datetime variables 
X['waiting_time'] =X['out_time'].sub(X['in_time'], axis=0)
X['waiting_time']= X['waiting_time'].dt.seconds

In [15]:
X.head(50)

,token,task,age,gender,dept,doctortype,in_time,out_time,dayofweek,is_weekday,session,age_category,waiting_time
0,1,checkup,15,f,General Medicine,physician,2019-12-28 10:00:00,2019-12-28 10:03:00,5,0,Morning,child,180
1,1,billing,15,f,cashier,unidentifed,2019-12-28 10:04:00,2019-12-28 10:05:00,5,0,Morning,child,60
2,1,medicine,15,f,pharmacy,unidentifed,2019-12-28 10:06:00,2019-12-28 10:08:00,5,0,Morning,child,120
3,2,checkup,21,m,Surgery,surgeon,2019-12-28 10:04:00,2019-12-28 10:08:00,5,0,Morning,adult,240
4,2,ct-scan,21,m,ct_lab,unidentifed,2019-12-28 10:09:00,2019-12-28 10:23:00,5,0,Morning,adult,840
5,2,billing,21,m,cashier,unidentifed,2019-12-28 10:13:00,2019-12-28 10:15:00,5,0,Morning,adult,120
6,2,medicine,21,m,pharmacy,unidentifed,2019-12-28 10:15:00,2019-12-28 10:17:00,5,0,Morning,adult,120
7,3,checkup,45,m,General Medicine,physician,2019-12-28 10:05:00,2019-12-28 10:09:00,5,0,Morning,adult,240
8,3,billing,45,m,cashier,unidentifed,2019-12-28 10:10:00,2019-12-28 10:11:00,5,0,Morning,adult,60
9,3,medicine,45,m,pharmacy,unidentifed,2019-12-28 10:12:00,2019-12-28 10:13:00,5,0,Morning,adult,60


In [16]:
#delete none essential columns
X.drop('out_time' ,axis=1, inplace=True)
X.drop('in_time' ,axis=1, inplace=True)
X.drop('dayofweek' ,axis=1, inplace=True)
X.drop('age' ,axis=1, inplace=True)

X.dtypes

token              int64
task              object
gender            object
dept              object
doctortype        object
is_weekday         int64
session         category
age_category    category
waiting_time       int64
dtype: object

In [17]:
X.head(100)

,token,task,gender,dept,doctortype,is_weekday,session,age_category,waiting_time
0,1,checkup,f,General Medicine,physician,0,Morning,child,180
1,1,billing,f,cashier,unidentifed,0,Morning,child,60
2,1,medicine,f,pharmacy,unidentifed,0,Morning,child,120
3,2,checkup,m,Surgery,surgeon,0,Morning,adult,240
4,2,ct-scan,m,ct_lab,unidentifed,0,Morning,adult,840
...,...,...,...,...,...,...,...,...,...
95,28,fracture,m,General Medicine,orthopedic,0,Evening,adult,1200
96,28,billing,m,cashier,unidentifed,0,Evening,adult,120
97,28,medicine,m,pharmacy,unidentifed,0,Evening,adult,120
98,29,checkup,f,General Medicine,physician,0,Evening,child,240


In [18]:
#convert string categories to integer for processing
category_col =['gender','dept','task','age_category','doctortype','session']  
labelEncoder = preprocessing.LabelEncoder() 
  
mapping_dict ={} 
for col in category_col: 
    X[col] = labelEncoder.fit_transform(X[col]) 
  
    le_name_mapping = dict(zip(labelEncoder.classes_, 
                        labelEncoder.transform(labelEncoder.classes_))) 
  
    mapping_dict[col]= le_name_mapping

In [19]:
X.head(59)

,token,task,gender,dept,doctortype,is_weekday,session,age_category,waiting_time
0,1,2,0,0,1,0,1,1,180
1,1,0,0,3,3,0,1,1,60
2,1,5,0,7,3,0,1,1,120
3,2,2,1,2,2,0,1,0,240
4,2,3,1,4,3,0,1,0,840
5,2,0,1,3,3,0,1,0,120
6,2,5,1,7,3,0,1,0,120
7,3,2,1,0,1,0,1,0,240
8,3,0,1,3,3,0,1,0,60
9,3,5,1,7,3,0,1,0,60


In [20]:
#variable on which prediction is done
y = X.iloc[:,8]
#new data frame consisting on essential factors for prediction
X1=X.drop(['token','waiting_time'],axis=1)

X1.head(10)

,task,gender,dept,doctortype,is_weekday,session,age_category
0,2,0,0,1,0,1,1
1,0,0,3,3,0,1,1
2,5,0,7,3,0,1,1
3,2,1,2,2,0,1,0
4,3,1,4,3,0,1,0
5,0,1,3,3,0,1,0
6,5,1,7,3,0,1,0
7,2,1,0,1,0,1,0
8,0,1,3,3,0,1,0
9,5,1,7,3,0,1,0


In [26]:
y.head(100)

0      180
1       60
2      120
3      240
4      840
      ... 
95    1200
96     120
97     120
98     240
99     120
Name: waiting_time, Length: 100, dtype: int64

In [30]:
model = RandomForestClassifier(10,random_state=42,oob_score=True)
model.fit(X1,y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [31]:
model.score(X1,y)

0.7542372881355932

In [34]:
#pass values for calculation 
t = model.predict([[2,1,0,1,0,1,2]])
t


array([300], dtype=int64)

In [35]:
#converting to seconds to minutes 
day = t//86400 
hour = (t-(day*86400))//3600 
min = (t - ((day*86400) + (hour*3600)))//60 
seconds = t - ((day*86400) + (hour*3600) + (min*60)) 

hello = datetime.time(hour, min, seconds)   

print (hello)

00:05:00
